## Import libraries

In [1]:
import fitz
import json
from re import findall
from re import sub
from os import listdir
from os.path import isfile, join
import PyPDF2
import glob
import en_core_web_sm
import numpy as np  
import pandas as pd
import re  
import nltk  
from sklearn.datasets import load_files 
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
import string
from operator import itemgetter


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdullah.kurkcu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Reading Resumes into DataFrame

Does it have education?
Does it have professional?
Does it have project experience?
Does it have professional associations?

In [2]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")

i = 0 # 81 is the start of not resume
styles = {}
for file in pdf_files:
    doc = fitz.open(file)
    for page in doc:
        blocks = page.getText("dict")["blocks"]
        fonts = []
        sizes = []
        colors = []
        images = []
        body = ''
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        body += " " + s['text'].lower()
                        fonts.append(s['font'])
                        sizes.append(s['size'])
                        colors.append(s['color'])
            if b['type'] == 1:  # block contains image
                images.append(1)
        if 'education' in body:
            ed_check = 1
        else:
            ed_check = 0
        if 'professional registrations' in body:
            pr_check = 1
        else:
            pr_check = 0
        if 'associations' in body:
            pa_check = 1
        else:
            pa_check = 0
        if 'experience' in body:
            pe_check = 1
        else:
            pe_check = 0
        if len(images)>0:
            img_check = 1
        else:
            img_check = 0
        
    if i < 81:
        label = 'resume'
    else:
        label = 'not resume'
    styles[doc] =  {'name':doc, 'body':body,'fonts':list(set(fonts)), 'size':list(set(sizes)),'color':list(set(colors)),'education':ed_check, 'registrations':pr_check,
                   'associations':pa_check, 'experience':pe_check, 'image':img_check, 'label':label}
    i = i + 1

data = pd.DataFrame.from_dict(styles, orient='index')

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object


In [3]:
#data['fonts'] = [','.join(map(str, l)) for l in df['fonts']]

data.head()

,name,body,fonts,size,color,education,registrations,associations,experience,image,label
(page 0 of ../resumes\17.00532 0.pdf),(page 0 of ../resumes\17.00532 0.pdf),15 buena vista / midland hills bridge trail...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 12.0, 14.0]","[16777215, 32117, 2236191]",1,1,1,1,1,resume
(page 0 of ../resumes\17.00532 1.pdf),(page 0 of ../resumes\17.00532 1.pdf),16 buena vista / midland hills bridge trail...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 12.0, 14.0]","[32117, 2236191]",1,1,0,1,1,resume
(page 0 of ../resumes\17.00532 2.pdf),(page 0 of ../resumes\17.00532 2.pdf),17 buena vista / midland hills bridge trail...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 12.0, 14.0]","[32117, 2236191]",1,1,0,1,1,resume
(page 0 of ../resumes\17.00532 3.pdf),(page 0 of ../resumes\17.00532 3.pdf),17 buena vista / midland hills bridge trail...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 12.0, 14.0]","[32117, 2236191]",1,1,0,1,1,resume
(page 0 of ../resumes\18.00308 0.pdf),(page 0 of ../resumes\18.00308 0.pdf),14 jefferson county open space / roadway si...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 12.0, 14.0]","[32117, 2236191]",1,1,1,1,1,resume


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics
import pandas as pd


# text and numeric classes that use sklearn base libaries
class TextTransformer(BaseEstimator, TransformerMixin):
    """
    Transform text features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberTransformer(BaseEstimator, TransformerMixin):
    """
    Transform numeric features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [6]:
df = data

vec_tdidf = TfidfVectorizer(ngram_range=(1,1), analyzer='word', norm='l2')

# compile both the TextTransformer and TfidfVectorizer 
# to the text 'Text_Feature' 
body_text = Pipeline([
                ('transformer', TextTransformer(key='body')),
                ('vectorizer', vec_tdidf)
                ])

ed_numeric = Pipeline([
                ('transformer', NumberTransformer(key='education')),
                ])
reg_numeric = Pipeline([
                ('transformer', NumberTransformer(key='registrations')),
                ])
assc_numeric = Pipeline([
                ('transformer', NumberTransformer(key='associations')),
                ])
exp_numeric = Pipeline([
                ('transformer', NumberTransformer(key='experience')),
                ])
img_numeric = Pipeline([
                ('transformer', NumberTransformer(key='image')),
                ])

# combine all of the features, text and numeric together
features = FeatureUnion([ ('body', body_text),
                          ('education', ed_numeric),
                          ('registrations', reg_numeric),
                          ('associations', assc_numeric),
                          ('experience', exp_numeric),
                          ('image', img_numeric)
                      ])

# create the classfier from RF
clf = RandomForestClassifier()

# unite the features and classfier together
pipe = Pipeline([('features', features),
                 ('clf',clf)
                 ])

# transform the categorical predictor into numeric
predicted_dummies = pd.get_dummies(df['label'])

# split the data into train and test
# isolate the features from the predicted field
text_numeric_features = ['body','education', 'registrations', 'associations', 'experience','image']
predictor = 'label'

X_train, X_test, y_train, y_test = train_test_split(df[text_numeric_features], df[predictor], 
                                                    test_size=0.25, random_state=42)

# fit the model
pipe.fit(X_train, y_train)

# predict from the test set
preds = pipe.predict(X_test)

# print out your accuracy
print("Accuracy:",metrics.accuracy_score(y_test, preds))

Accuracy: 0.9142857142857143


### Page Reader

In [11]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumesnot/")

i = 0 # 81 is the start of not resume
styles = {}
for file in pdf_files:
    doc = fitz.open(file)
    for page in doc:
        blocks = page.getText("dict")["blocks"]
        fonts = []
        sizes = []
        colors = []
        images = []
        body = ''
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        body += " " + s['text'].lower()
                        fonts.append(s['font'])
                        sizes.append(s['size'])
                        colors.append(s['color'])
            if b['type'] == 1:  # block contains image
                images.append(1)
        if 'education' in body:
            ed_check = 1
        else:
            ed_check = 0
        if 'professional registrations' in body:
            pr_check = 1
        else:
            pr_check = 0
        if 'associations' in body:
            pa_check = 1
        else:
            pa_check = 0
        if 'experience' in body:
            pe_check = 1
        else:
            pe_check = 0
        if len(images)>0:
            img_check = 1
        else:
            img_check = 0

        styles[page] =  {'name':doc, 'body':body,'fonts':list(set(fonts)), 'size':list(set(sizes)),'color':list(set(colors)),'education':ed_check, 'registrations':pr_check,
                   'associations':pa_check, 'experience':pe_check, 'image':img_check, 'label':label}
    i = i + 1

pdf_data = pd.DataFrame.from_dict(styles, orient='index')

In [12]:
pdf_data

,name,body,fonts,size,color,education,registrations,associations,experience,image,label
page 0 of ../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf,(page 0 of ../resumesnot\19.00150_Town of Vail...,proposal to perform professional engineering ...,"[NewsGothicStd-Bold, NewsGothicStd]","[10.0, 11.199999809265137, 12.0, 15.0]",[16777215],0,0,0,0,1,not resume
page 1 of ../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf,(page 0 of ../resumesnot\19.00150_Town of Vail...,ulteig denver office 5575 dtc parkway suite 2...,"[ArialMT, NewsGothicStd-Bold, NewsGothicStd]","[217.53359985351562, 10.0, 6.0]","[0, 5000266, 16777215]",0,0,0,1,1,not resume
page 2 of ../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf,(page 0 of ../resumesnot\19.00150_Town of Vail...,5 project approach 9 firm & experienc...,"[NewsGothicStd, NewsGothicStd-Bold, ArialMT]","[69.81995391845703, 6.0, 10.0, 13.0, 217.53359...","[0, 5000266, 16777215]",0,0,0,1,1,not resume
page 3 of ../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf,(page 0 of ../resumesnot\19.00150_Town of Vail...,town of vail / gore valley trail design imp...,"[NewsGothicStd-Bold, NewsGothicStd-Oblique, Ne...","[8.0, 10.0]","[0, 32117]",0,0,0,0,1,not resume
page 4 of ../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf,(page 0 of ../resumesnot\19.00150_Town of Vail...,town of vail / gore valley trail design imp...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 20.0]","[0, 32117, 16777215]",0,0,0,0,1,not resume
...,...,...,...,...,...,...,...,...,...,...,...
page 18 of ../resumesnot\21.00980 Sioux City Proposal.pdf,(page 0 of ../resumesnot\21.00980 Sioux City P...,downtown infrastructure reconstruction maste...,"[Calibri-Bold, ArialNarrow-Bold, ArialNarrow]","[9.119999885559082, 10.079999923706055, 11.039...","[0, 16711680, 14687781, 2104606]",1,1,0,1,1,not resume
page 19 of ../resumesnot\21.00980 Sioux City Proposal.pdf,(page 0 of ../resumesnot\21.00980 Sioux City P...,downtown infrastructure reconstruction maste...,"[Calibri-Bold, ArialNarrow-Bold, ArialNarrow, ...","[9.119999885559082, 10.079999923706055, 11.039...","[0, 16711680, 14687781, 32118, 2104606]",1,1,0,1,1,not resume
page 20 of ../resumesnot\21.00980 Sioux City Proposal.pdf,(page 0 of ../resumesnot\21.00980 Sioux City P...,downtown infrastructure reconstruction maste...,"[ArialNarrow-Bold, Arial-BoldMT, Calibri-Bold,...","[10.079999923706055, 11.039999961853027, 13.92...","[0, 16711680, 32118]",0,0,0,0,1,not resume
page 21 of ../resumesnot\21.00980 Sioux City Proposal.pdf,(page 0 of ../resumesnot\21.00980 Sioux City P...,downtown infrastructure reconstruction maste...,"[ArialNarrow-Bold, Arial-BoldMT, Calibri-Bold,...","[10.079999923706055, 11.039999961853027, 13.92...","[0, 16711680, 32118]",0,0,0,1,1,not resume


In [13]:
# predict from the test set
preds = pipe.predict(pdf_data)

# print out your accuracy
#print("Accuracy:",metrics.accuracy_score(y_test, preds))

In [14]:
pdf_data['preds'] = preds
pdf_data[pdf_data['preds'] == 'resume']

,name,body,fonts,size,color,education,registrations,associations,experience,image,label,preds
page 11 of ../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf,(page 0 of ../resumesnot\19.00150_Town of Vail...,town of vail / gore valley trail design imp...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 11.0, 12.0]","[0, 32117, 16777215]",0,0,0,1,1,not resume,resume
page 13 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,section d: description of similar project...,"[NewsGothicStd-Bold, ArialNarrow]","[16.0, 10.5, 12.0]","[2373188, 13027530, 16296473, 811870, 2236191]",0,0,0,1,1,not resume,resume
page 16 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,key staff bios & project team resumes abdulla...,"[ArialNarrow-Bold, NewsGothicStd-Bold, ArialNa...","[10.5, 12.0]","[16777215, 2236191]",0,0,0,1,1,not resume,resume
page 17 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"will kerns, aicp | senior transportation pla...","[CIDFont+F3, CIDFont+F2, ArialNarrow, CIDFont+F4]","[8.982749938964844, 9.940999984741211, 8.98274...","[0, 2576241, 16777215]",1,1,1,1,1,not resume,resume
page 18 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"will kerns, aicp | senior transportation pla...","[CIDFont+F3, CIDFont+F4, ArialNarrow, CIDFont+...","[8.982749938964844, 9.94124984741211, 10.02000...","[0, 2576241]",1,0,0,0,1,not resume,resume
page 19 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,jennifer turnbow | senior technical manager...,"[ArialNarrow-Bold, SymbolMT, ArialNarrow]","[9.0, 10.020000457763672, 19.979999542236328, ...","[0, 2576241, 16777215]",1,0,1,1,1,not resume,resume
page 20 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,jennifer turnbow | senior technical manager...,"[ArialNarrow-Bold, SymbolMT, ArialNarrow]","[9.0, 10.020000457763672]","[0, 2576241]",0,0,0,0,1,not resume,resume
page 21 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"abdullah kurkcu, phd | lead traffic modele...","[ArialNarrow-Bold, Calibri, SymbolMT, ArialNar...","[9.0, 10.020000457763672, 10.979999542236328, ...","[0, 2576241, 16777215]",1,0,0,1,1,not resume,resume
page 22 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"abdullah kurkcu, phd | lead traffic modele...","[ArialNarrow-Bold, Calibri, SymbolMT, ArialNar...","[9.0, 10.979999542236328, 10.020000457763672]","[0, 2576241, 16777215]",0,0,1,1,1,not resume,resume
page 23 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"paul deutsch, pe | engineer page 1 paul de...","[ArialNarrow-Bold, SymbolMT, ArialNarrow]","[9.0, 10.020000457763672, 9.960000038146973, 1...","[0, 2576241, 16777215]",1,1,1,1,1,not resume,resume


#### Notes
Since it transforms fonts and sizes to columns, new documents will not have the same amount of columns that the test data will have

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

# Using Decision Tree Classifier on the data
dtclf = tree.DecisionTreeClassifier()
dtclf = dtclf.fit(resumes_train, y_train)
print(dtclf.score(resumes_test, y_test))


# Using Random Forest Classifier on the data
rfclf = RandomForestClassifier()
rfclf = rfclf.fit(resumes_train, y_train)
print(rfclf.score(resumes_test, y_test))

# Using SVM Classifier on the data
model_svm = svm.SVC()
model_svm = model_svm.fit(resumes_train, y_train)
print(model_svm.score(resumes_test, y_test))

# Using Bernoulli Naive Bayes Algorithm
bnbclf = BernoulliNB()
bnbclf = bnbclf.fit(resumes_train, y_train)
print(bnbclf.score(resumes_test, y_test))

# Using Gaussian Naive Bayes Algorithm
gnbclf = GaussianNB()
gnbclf = gnbclf.fit(resumes_train, y_train)
print(gnbclf.score(resumes_test, y_test))

0.9743589743589743
1.0
0.9743589743589743
0.9743589743589743
1.0


## Convert text into a vector 

In [99]:
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))


# Do not call fit transform twice because it is stored in vectorizer 
X_train = vectorizer.fit_transform(resumes_train)
X_test = vectorizer.transform(resumes_test)
y_train = y_train

AttributeError: 'float' object has no attribute 'lower'

In [8]:
X_train

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

## Train Classifier

In [7]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
text_clf  = classifier.fit(X_train, y_train) 

ValueError: Number of labels=78 does not match number of samples=10

In [98]:
file = '../downloaded/20.02373 CDOT NPS Western Slope_Ulteig.pdf'
fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
count = fileReader.numPages


results = text_clf.predict(X_test)
resumesTest['results'] = results
results

array(['NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume'],
      dtype=object)

In [70]:
text = ""
i = 0
X_test = []
while i < count:
    text = fileReader.getPage(i).extractText()
    X_test.append(text)
    
    i = i +1  



X_test = vectorizer.transform(X_test)

## Make a prediction

In [65]:
resumesTest.loc[0]['text']

'proposal to perform professional engineering services for  gore valley trail design improvements prepared for town of vail angy casamento peangycasamentoulteigcom       february  '

### Other

In [52]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")


# Resumes -> convert to a dictionary with an employee name
resumes = pd.DataFrame()

texts = []
# File loop
for file in pdf_files:
    print(file)
    text = ""
    try:
        fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
        count = fileReader.numPages
        text = ""
        i = 0
        while i < count:
            text += fileReader.getPage(i).extractText()
            i = i +1  
        # lowercase
        text = text.lower()
        # remove nubmers
        text = re.sub(r'\d+', '', text)
        # remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # remove \n
        text = text.replace('\n',' ')
        # remove ndł
        text= text.replace('ndł',' ')
        # remove \nł
        text = text.replace('\nł',' ')
        # remove ł
        text = text.replace('ł',' ')
        # remove \nł
        text = text.replace('\nł',' ')

        texts.append(text)
    except Exception as e:
        print(e)
        pass
    
resumes['text'] = texts
resumes['classification'] = "Resume"

../resumes\17.00532 0.pdf
../resumes\17.00532 1.pdf
../resumes\17.00532 2.pdf
../resumes\17.00532 3.pdf
../resumes\18.00308 0.pdf
../resumes\18.00308 1.pdf
../resumes\18.00308 2.pdf
../resumes\18.00308 3.pdf
../resumes\19.00064 0.pdf
../resumes\19.00064 1.pdf
../resumes\19.00064 10.pdf
../resumes\19.00064 11.pdf
../resumes\19.00064 2.pdf
../resumes\19.00064 3.pdf
../resumes\19.00064 4.pdf
../resumes\19.00064 5.pdf
../resumes\19.00064 6.pdf
../resumes\19.00064 7.pdf
../resumes\19.00064 8.pdf
../resumes\19.00064 9.pdf
../resumes\20.00432 0.pdf
../resumes\20.00432 1.pdf
../resumes\20.00432 2.pdf
../resumes\20.00432 3.pdf
../resumes\20.00432 4.pdf
../resumes\20.00973 0.pdf
../resumes\20.00973 1.pdf
../resumes\20.00973 10.pdf
../resumes\20.00973 11.pdf
../resumes\20.00973 12.pdf
../resumes\20.00973 2.pdf
../resumes\20.00973 3.pdf
../resumes\20.00973 4.pdf
../resumes\20.00973 5.pdf
../resumes\20.00973 6.pdf
../resumes\20.00973 7.pdf
../resumes\20.00973 8.pdf
../resumes\20.00973 9.pdf
../resu

In [185]:
from operator import itemgetter
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}
                        else:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}

    return styles

### OpenCV trials

In [1]:
from pdf2image import convert_from_path

file = '../downloaded/20.02373 CDOT NPS Western Slope_Ulteig.pdf'
images = convert_from_path(file)

In [6]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")

# File loop
i = 0 
for file in pdf_files:
    images = convert_from_path(file)
    for image in images:
        filename = "resume" + str(i) + ".jpg"
        image.save(filename, 'JPEG')
        i = i +1